In [8]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd
import requests

# Sample data
data = pd.DataFrame({
    'Country': ['Switzerland', 'Germany', 'United States', 'China', 'Italy', 'France', 'United Kingdom', 'India', 'Japan', 'Netherlands', 'Belgium', 'Austria', 'Spain', 'Sweden', 'Poland', 'Hong Kong', 'Singapore', 'Russia', 'South Korea', 'Brazil', 'Mexico'],
    'ISO_Code': [
        'CHE', 'DEU', 'USA', 'CHN', 'ITA', 'FRA', 'GBR', 'IND', 'JPN', 'NLD',
        'BEL', 'AUT', 'ESP', 'SWE', 'POL', 'HKG', 'SGP', 'RUS', 'KOR', 'BRA', 'MEX'
    ],
    'Attractiveness': [90, 70, 80, 75, 60, 85, 70, 50, 40, 50, 60, 50, 40, 30, 50, 40, 30, 50, 60, 70, 80],
    'Economy': [8.5, 7.0, 8.0, 7.5, 6.0, 8.3, 8.0, 5.0, 3.0, 2.0, 5.7, 4.6, 7.0, 3.5, 6.0, 4.0, 3.5, 6.0, 6.0, 5.0, 4.5],
    'Stability': [9.0, 8.2, 5.8, 7.8, 5.5, 8.0, 6.0, 8.5, 8.0, 7.5, 6.0, 8.3, 8.0, 5.0, 3.0, 2.0, 5.7, 4.6, 7.0, 3.5, 6.0]
})

# World Bank API
def get_latest_unemployment(country_code):
    iso_map = {
        "CHE": "CH", "DEU": "DE", "USA": "US", "CHN": "CN", "ITA": "IT", "FRA": "FR",
        "GBR": "GB", "IND": "IN", "JPN": "JP", "NLD": "NL", "BEL": "BE", "AUT": "AT",
        "ESP": "ES", "SWE": "SE", "POL": "PL", "HKG": "HK", "SGP": "SG", "RUS": "RU",
        "KOR": "KR", "BRA": "BR", "MEX": "MX"
    }
    if country_code not in iso_map:
        return "Data not available"

    indicator = "SL.UEM.TOTL.ZS"
    url = f"https://api.worldbank.org/v2/country/{iso_map[country_code]}/indicator/{indicator}?format=json&per_page=1"

    try:
        r = requests.get(url, timeout=5)
        r.raise_for_status()
        data = r.json()
        if data and len(data) > 1 and data[1]:
            latest = data[1][0]
            value = latest['value']
            date = latest['date']
            return f"{value:.1f}% ({date})" if value is not None else "No data"
        else:
            return "No data"
    except Exception:
        return "Data not available"

# Dash app
app = dash.Dash(__name__, suppress_callback_exceptions=True)
app.title = "International Investment"

# Initial map figure
initial_fig = px.choropleth(
    data,
    locations="ISO_Code",
    color="Attractiveness",
    hover_name="Country",
    color_continuous_scale=px.colors.sequential.Plasma
)
initial_fig.update_layout(height=500, margin={"r":0,"t":0,"l":0,"b":0})

# Layout
app.layout = html.Div([
    html.H1("International Investment", style={'textAlign': 'center'}),
    dcc.Location(id='url', refresh=False),

    html.Div([
        html.A("Map", href="/", style={'margin-right': '20px'}),
        html.A("Comparator", href="/compare")
    ], style={'textAlign': 'center', 'margin-bottom': '30px'}),

    dcc.Graph(id='map', figure=initial_fig, style={'display': 'none'}),
    html.Div(id='page-content')
])

# Page navigation
@app.callback(
    Output('page-content', 'children'),
    Output('map', 'style'),
    Input('url', 'pathname')
)
def display_page(pathname):
    if pathname == "/compare":
        return html.Div([
            html.H2("Country Comparator"),
            dcc.Dropdown(
                id='countries',
                options=[{'label': c, 'value': c} for c in data['Country']],
                multi=True,
                placeholder="Select 2-5 countries"
            ),
            html.Div(id='compare-graphs')
        ]), {'display': 'none'}

    elif pathname.startswith("/country-"):
        code = pathname.split("-")[1]
        row = data[data['ISO_Code'] == code].iloc[0]
        return html.Div([
            html.H2(row['Country']),
            html.Div([
                html.Button("General", id='btn-general', n_clicks=0),
                html.Button("Economy", id='btn-economy', n_clicks=0),
                html.Button("Legal", id='btn-legal', n_clicks=0),
                html.Button("Competition", id='btn-competition', n_clicks=0)
            ], style={'margin-bottom':'20px'}),
            html.Div(id='country-subpage'),
            html.Br(),
            html.A("Back to map", href="/")
        ]), {'display': 'none'}

    else:
        return html.Div([
            html.H2("Global Map")
        ]), {'display': 'block'}

# Country page sub-tabs
@app.callback(
    Output('country-subpage', 'children'),
    Input('btn-general', 'n_clicks'),
    Input('btn-economy', 'n_clicks'),
    Input('btn-legal', 'n_clicks'),
    Input('btn-competition', 'n_clicks'),
    Input('url', 'pathname')
)
def update_country_subpage(n1, n2, n3, n4, pathname):
    code = pathname.split("-")[1]
    row = data[data['ISO_Code'] == code].iloc[0]

    ctx = dash.callback_context
    if not ctx.triggered:
        tab = "general"
    else:
        tab_id = ctx.triggered[0]['prop_id'].split('.')[0]
        tab = tab_id.split('-')[1]

    if tab == "general":
        return html.Div([
            html.H3("General Info"),
            html.P("No description available.")
        ])
    elif tab == "economy":
        unemployment = get_latest_unemployment(code)
        return html.Div([
            html.H3("Economy"),
            html.Ul([
                html.Li(f"Economy score: {row['Economy']}"),
                html.Li(f"Attractiveness: {row['Attractiveness']}"),
                html.Li(f"Unemployment Rate: {unemployment}")
            ])
        ])
    elif tab == "legal":
        return html.Div([
            html.H3("Legal"),
            html.P("Legal info placeholder")
        ])
    elif tab == "competition":
        return html.Div([
            html.H3("Competition"),
            html.P("Competition info placeholder")
        ])
    return html.Div("No data")

# Map click → country page
@app.callback(
    Output('url', 'pathname'),
    Input('map', 'clickData'),
    prevent_initial_call=True
)
def country_click(clickData):
    if clickData:
        code = clickData['points'][0]['location']
        return f"/country-{code}"
    return "/"

# Multi-country comparator
@app.callback(
    Output('compare-graphs', 'children'),
    Input('countries', 'value')
)
def update_compare(selected_countries):
    if not selected_countries or len(selected_countries) < 2:
        return html.P("Please select at least 2 countries.")
    if len(selected_countries) > 5:
        return html.P("You can select up to 5 countries only.")

    rows = data[data['Country'].isin(selected_countries)]
    graphs = []
    for col in ['Attractiveness', 'Economy', 'Stability']:
        fig = px.bar(rows, x='Country', y=col, color='Country', text=col)
        fig.update_layout(
            height=250,
            margin={"r": 10, "t": 30, "l": 10, "b": 10},
            showlegend=False
        )
        graphs.append(
            dcc.Graph(
                figure=fig,
                style={'width': '70%', 'margin': 'auto'}
            )
        )
    return graphs

# Run app
if __name__ == '__main__':
    app.run(debug=True)


In [2]:
print("et voici un nouvel essai hehe")

et voici un nouvel essai hehe
